In [36]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import re
import multiprocessing
from collections import Counter
import itertools
import json
from IPython.display import display as disp

import numpy as np
# import bottleneck as bn
import pandas as pd
import networkx as nx
# import igraph as ig
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='paper', style='whitegrid', color_codes=True, font_scale=1.8)
colorcycle = [(0.498, 0.788, 0.498),
              (0.745, 0.682, 0.831),
              (0.992, 0.753, 0.525),
              (0.220, 0.424, 0.690),
              (0.749, 0.357, 0.090),
              (1.000, 1.000, 0.600),
              (0.941, 0.008, 0.498),
              (0.400, 0.400, 0.400)]
%matplotlib inline
sns.set_palette(colorcycle)
mpl.rcParams['figure.max_open_warning'] = 65
mpl.rcParams['figure.figsize'] = [12, 7]

# import warnings
# warnings.simplefilter("ignore", category=DeprecationWarning)
# warnings.simplefilter("ignore", category=mpl.cbook.mplDeprecation)
# warnings.simplefilter("ignore", category=UserWarning)


from speclib import misc, loaders, graph, plotting

pd.set_option('display.max_rows', 55)
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 1000)
np.set_printoptions(linewidth=145)

import pickle
import warnings

# import pixiedust

In [38]:
warnings.filterwarnings('ignore', category=UserWarning)

In [39]:
def read_pkl(pkl):
    with open(pkl, 'br') as fid:
        data = pickle.load(fid)
    return data 

In [78]:
!/lscr_paper/allan/bin_allan/exa --header --long *.pickle *.h5 *.pk

Permissions Size User   Date Modified Name
.rw-rw-r--  146k bzg778 22 Mar  2017  clique_pca_evr_dump.pickle
.rw-rw-r--  142M bzg778 30 May  2017  ica_result_clique.pickle
.rw-rw-r--   13M bzg778 30 May  2017  ica_result_community.pickle
.rw-rw-r--   53M bzg778  1 Nov 13:19  parallel_pca_data.h5
.rw-rw-r--   63M bzg778 24 May  2017  pca_result_clique.pickle
.rw-rw-r--  5.0M bzg778 24 May  2017  pca_result_community.pickle
.rw-rw-r--  5.9M bzg778  1 Nov 13:19  temp.h5
.rw-rw-r--   57k bzg778 20 Feb  2017  useralias.pk


In [61]:
data = read_pkl('clique_pca_evr_dump.pickle') 
print(type(data))
print(len(data)) 

<class 'dict'>
5


In [76]:
data = read_pkl('ica_result_clique.pickle')
print(type(data))
print(data[0])
print(len(data))

<class 'list'>
(['u0489', 'u0128', 'u0491', 'u0813', 'u0287', 'u0758'], FastICA(algorithm='parallel', fun='logcosh', fun_args=None, max_iter=1500,
    n_components=None,
    random_state=<mtrand.RandomState object at 0x7fd3f3cecdc8>, tol=0.001,
    w_init=None, whiten=True))
411


In [75]:
data = read_pkl('ica_result_community.pickle')
print(type(data))
print(data[0]) 
print(len(data))

<class 'list'>
(['u0128', 'u0491', 'u0324', 'u1023', 'u0441', 'u0758', 'u0287', 'u0154', 'u0489', 'u0444', 'u0813', 'u0951', 'u0477', 'u0150'], FastICA(algorithm='parallel', fun='logcosh', fun_args=None, max_iter=1500,
    n_components=None,
    random_state=<mtrand.RandomState object at 0x7fd3f3c26948>, tol=0.001,
    w_init=None, whiten=True))
8


In [74]:
data = read_pkl('pca_result_clique.pickle')
print(type(data))
print(data[0]) 
print(len(data)) 

<class 'list'>
(['u0489', 'u0491', 'u0758', 'u0287', 'u0128', 'u0813'], PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False))
411


In [73]:
data = read_pkl('pca_result_community.pickle')
print(type(data))
print(data[0])
print(len(data))

<class 'list'>
(['u1023', 'u0154', 'u0287', 'u0951', 'u0491', 'u0324', 'u0444', 'u0150', 'u0477', 'u0128', 'u0813', 'u0758', 'u0441', 'u0489'], PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False))
8


In [70]:
with pd.HDFStore('parallel_pca_data.h5') as store:
    print(store.keys())
    print(store['communityLst'])
    disp(store['df'].head()) 

['/communityLst', '/df']
0    (u0324, u0287, u0813, u0491, u0758, u0128)
1           (u0324, u0287, u0477, u0444, u0128)
2                  (u0786, u1040, u0808, u0930)
3                         (u0111, u0147, u0576)
dtype: object


user comtype body  duration  hour     timeint           timestamp  weekday contactedUser  tbin
user  comtype                                                                                                 
u0001 call     u0001    call  NaN       0.0     1  1390698006 2014-01-26 01:00:06        6         u0493   474
      call     u0001    call  NaN       0.0    14  1391176805 2014-01-31 14:00:05        4         u0493   490
      call     u0001    call  NaN      29.0    14  1391782631 2014-02-07 14:17:11        4         u0117   511
      call     u0001    call  NaN      22.0    14  1391782712 2014-02-07 14:18:32        4         u0493   511
      call     u0001    call  NaN       3.0    14  1394805935 2014-03-14 14:05:35        4         u0117   616

In [72]:
with pd.HDFStore('temp.h5') as store:
    print(store.keys()) 
    disp(store['dftmp'].head()) 
    print(store['dftmp'].shape)

['/(u0841, u0233, u0748, u0404, u0788)', '/dftmp']


,0
"(u0645, u0683, u0618, u0388, u0676, u0993, u0446)","PCA(copy=True, iterated_power='auto', n_compon..."
"(u0241, u0392, u0122, u0478, u0739, u0264)","PCA(copy=True, iterated_power='auto', n_compon..."
"(u0324, u0973, u0818, u0758, u0441)","PCA(copy=True, iterated_power='auto', n_compon..."
"(u0149, u0011, u0768, u0202)","PCA(copy=True, iterated_power='auto', n_compon..."
"(u0715, u0473, u0122)","PCA(copy=True, iterated_power='auto', n_compon..."


(5, 1)


In [84]:
data = read_pkl('useralias.pk')
print(type(data))

<class 'speclib.loaders.Useralias'>
